In [ ]:
from gurobipy import *
from gurobipy import Model, GRB, quicksum
import pandas as pd
import random
import threading
from random import randint
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import operator
import os.path
import time
import copy

In [ ]:
lst = [
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/New_Dataset_CVRPTW/20/20200928_114457.csv",
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/New_Dataset_CVRPTW/20/20200925_093755.csv"
]

lst_distance = [
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/New_Dataset_CVRPTW/20/20200928_114457_dist.csv",
    "C:/Users/German/Desktop/Курсовая_работа/Алгоритмы/Датасеты/New_Dataset_CVRPTW/20/20200925_093755_dist.csv"
]

In [ ]:
%%time

for name_file1, name_file2 in zip(lst, lst_distance):
    data_file    = pd.read_csv(name_file1, sep="\t", error_bad_lines=True)
    
    # Запишем все данные в виде массива списков, где каждый список это строка с значениями конкретных столбцов
    arr_data     = data_file.values

    # Задаём объем грузовика
    Q_max = 500 # Для 20:500, 50:750, 100:1000
    
    # Создаём списки координат и грузов для каждого клиента
    xc = np.zeros(len(arr_data))
    yc = np.zeros(len(arr_data))
    capacity = np.zeros(len(arr_data))
    w = 0
    for i in range(len(arr_data)):
        if(arr_data[i][2] <= Q_max):
            xc[w]       = arr_data[i][0]
            yc[w]       = arr_data[i][1]
            capacity[w] = arr_data[i][2]
            w += 1

    # Убираем координаты и вес депо
    depo_xc = xc[w-1]
    depo_yc = yc[w-1]
    xc = xc[:w-1]
    yc = yc[:w-1]
    capacity = capacity[:w-1]
    
    # Исключаем депо из списка клиентов
    w = w - 1
    
    # Создаём список клиентов
    clients  = []
    for i in range(0, w):
        clients.append(i)
    # Cоздаем список клиентов вместе с депо. Депо обозначено, как последний элемент в arr_data
    nodes   = [len(arr_data) - 1] + clients
    # Создаём список всевохможных пар между клиентами, включая депо
    paires  = [(i, j) for i in nodes for j in nodes if i != j]
    
    # Создаём словарь весов
    q                    = {i: capacity[i] for i in clients}
    q[len(arr_data) - 1] = 0 # добавляем вес в депо в конец словаря
    
    #Задаём словари временных ограничений на посещение
    
    # Минимальное время прибытия + перевод в минуты
    start_lst  = {i: int(arr_data[i][3][:2]) * 60 for i in clients}
    start      = {len(arr_data) - 1: int(arr_data[19][3][:2]) * 60} # временное окно для депо
    start.update(start_lst)

    # Максимальное время прибытия + перевод в минуты
    finish_lst = {i: int(arr_data[i][3][6:8]) * 60 for i in clients}
    finish     = {len(arr_data) - 1: int(arr_data[19][3][6:8]) * 60} # временное окно для депо
    finish.update(finish_lst)
    
    # Задаём словарь времени обслуживания каждого клиента (в минутах)
    service                    = {n: int(arr_data[n][4]) for n in clients}
    # Добавляем в словарь время обслуживания в депо
    service[len(arr_data) - 1] = 0

    #Список транспортных средств
    vehicles = [1]
    
    #Словарь вместимости транспортных средств
    Q        = {1: Q_max}
    
    #Задаем скорость грузовика 60 км/ч = 1000 м/мин
    v = 1000
    
    # Задаем словарь всевозможных расстояний между всеми городами, включая депо
    # Расстояния выражены в МЕТРАХ
    distance  = {}
    arr_dist  = pd.read_csv(name_file2, sep=",", error_bad_lines=False)
    for i,j in paires:    
        if(i == len(arr_data) - 1 and j <= max(paires)[1]):
            for k in range(len(arr_dist)):
                if(depo_xc == arr_dist.values[k][0] and depo_yc == arr_dist.values[k][1] and xc[j] == arr_dist.values[k][2] and yc[j] == arr_dist.values[k][3]):
                    distance[(i,j)] = arr_dist.values[k][4]
                    break

        elif(i <= max(paires)[1] and j == len(arr_data) - 1):
             for k in range(len(arr_dist)):
                if(xc[i] == arr_dist.values[k][0] and yc[i] == arr_dist.values[k][1] and depo_xc == arr_dist.values[k][2] and depo_yc == arr_dist.values[k][3]):
                    distance[(i,j)] = arr_dist.values[k][4]
                    break

        else:
             for k in range(len(arr_dist)):
                if(xc[i] == arr_dist.values[k][0] and yc[i] == arr_dist.values[k][1] and xc[j] == arr_dist.values[k][2] and yc[j] == arr_dist.values[k][3]):
                    distance[(i,j)] = arr_dist.values[k][4]
                    break
    
    # Задаем словарь времени, требуемое для перемещение из одного города в другой
    # Время выражено в МИНУТАХ
    time = {}
    for i,j in paires:
        time[(i, j)] = distance[i, j] / v
    
    arco_var = [(i, j, k) for i in nodes for j in nodes for k in vehicles if i != j]

    arco_time = [(i, k) for i in nodes for k in vehicles]
    print(arco_time)
    # Модель
    model = Model('VRPTW')

    # Переменные для решения
    x = model.addVars(arco_var, vtype=GRB.BINARY, name = 'x')
    t = model.addVars(arco_time, vtype=GRB.CONTINUOUS, name = 't')

    #Целевая функция
    model.setObjective(quicksum(distance[i, j]*x[i,j,k] for i, j, k in arco_var), GRB.MINIMIZE)

    #Ограничения
    
    # Прибытие и отъезд
    model.addConstrs(quicksum(x[len(arr_data) - 1, j, k] for j in clients) <= 1 for k in vehicles)
    model.addConstrs(quicksum(x[i, len(arr_data) - 1, k] for i in clients) <= 1 for k in vehicles)
    
    # Ограничение по посещению маршрута ровно один раз и ровно одним грузовиком
    model.addConstrs(quicksum(x[i, j, k] for j in nodes for k in vehicles if i != j) == 1 for i in clients)
    
    #
    model.addConstrs(quicksum(x[i, j, k] for j in nodes if i != j) - quicksum(x[j, i, k] for j in nodes if i != j) == 0 for i in nodes for k in vehicles)

    #Ограничение на вместимость грузовика
    model.addConstrs(quicksum(q[i] * quicksum(x[i, j, k] for j in nodes if i != j) for i in clients) <= Q[k] for k in vehicles)

    #Ограничение по времени
    model.addConstrs((x[i,j,k] == 1) >> (t[i, k] + service[i] + time[i, j] == t[j, k]) for i in clients for j in clients for k in vehicles if i != j)
    model.addConstrs(t[i, k] >= start[i] for i, k in arco_time)
    model.addConstrs(t[i, k] <= finish[i] for i, k in arco_time)

    # Optimizing the model
    model.Params.TimeLimit = 200  # seconds
    model.Params.LogFile= "Gurobi_20.txt"
    model.optimize()
    if model.status == GRB.OPTIMAL:
        print('1.Optimal objective: %g' % model.objVal)
        print('Optimal cost: %g' % model.objVal)
    elif model.status == GRB.INFEASIBLE:
        print('3.Model is infeasible')
        res = -1
    elif model.status == GRB.UNBOUNDED:
        print('4.Model is unbounded')
        res = -1
    else:
        print('5.Optimization ended with status %d' % model.status)
        print('Optimal cost: %g' % model.objVal)